# Mumbai legacy linelist data: Clean up 

Clean up these types of columns:
- Enums: dump out a list of enum maps, get the canonical values for these and use these enum maps to replace column values
- Dates: Use datefinder to get dates
- Regex: Use regex for remaining columns
- Text (todo, may/may not be required)

Assign status to each cell after clean up:

Enum/regex columns
- GOOD_PARSE: Clean
- MISSING_VAL: Value is missing/NA
- BAD_PARSE: Needs manual review

Date columns
- GOOD_PARSE: Clean
- MISSING_PARSE: Date is missing/NA
- BAD_PARSE: Cannot be parsed, needs manual review
- SUSPECTED_PARSE: Ambiguous as it can be parsed as MM/DD or DD/MM, further date processing for suspected dates in another notebook

Assign overall status to row as:
- GOOD: All values are either GOOD_PARSE or MISSING_VAL
- BAD: At least one BAD_PARSE but no SUSPECTED_PARSE
- SUSPECTED: Contains at least one SUSPECTED_PARSE

Execution: Read data file, set columns (Column Categories section) and dump out enum maps (Creating enum maps section). Get enum maps manually filled up. Set path to enum maps and run remaining parts.

In [ ]:
import pandas as pd

import datetime as dt
import datefinder
from dateutil.parser import parse
from datetime import timedelta
import time

from stemming.porter2 import stem
import re

## Input Files

In [ ]:
# Input files
MAP_DIR = '../maps/' # Directory containing enum map CSVs
DATA_FILE = '../data/consolidated-wards-new.csv'

# Map to canonical values
GENDER_LIST_FILE =  MAP_DIR + "GENDER_LIST_FILE.csv"
WARD_LIST_FILE =  MAP_DIR + "WARD_LIST_FILE.csv"
DISTRICT_LIST_FILE =  MAP_DIR +"DISTRICT_LIST_FILE.csv"
FACILITY_LIST_FILE =  MAP_DIR +"FACILITY_LIST_FILE.csv"
OUTCOME_LIST_FILE =  MAP_DIR +"OUTCOME_LIST_FILE.csv"
HSTATUS_LIST_FILE =  MAP_DIR +"HSTATUS_LIST_FILE.csv"
INTERVENTION_LIST_FILE =  MAP_DIR +"INTERVENTION_LIST_FILE.csv"
RESULT_STATUS_LIST_FILE =  MAP_DIR + "RESULT_STATUS_LIST_FILE.csv"
BOOL_LIST_FILE =  MAP_DIR + "BOOL_LIST_FILE.csv"


## Reading the data

In [ ]:
D = pd.read_csv(DATA_FILE, header=0, low_memory=False)

In [ ]:
D.shape

In [ ]:
D.columns

In [ ]:
D.head()

##  Parsing routines

In [ ]:
def check_na(x):
    """Check if a cell value is NA. 
    This includes the NaN, hyphens, whitespace and strings such as [na, nan, n/a]."""
    if pd.isna(x):
        return True
    if isinstance(x, str):
        if re.match('(-)+', x) or x.isspace() or x.strip().lower() in ["na", "nan", "n/a"]:
            return True
    return False

In [ ]:
# Simple date parsing
 
def order_fix(x,valid_interval): # NOT USED
    """mm-dd-order fix """
    valid_start = valid_interval[0]
    valid_end = valid_interval[1]
    if ((x > valid_end) or (x < valid_start)):
        x =dt.date(x.year,x.day,x.month)
    if ((x > valid_end) or (x < valid_start)):
        x =None    
    return x

def parse_date_old(x,valid_interval): # NOT USED
    status ="GOOD_PARSE"
    result = x
    if not result:
        result = "NA"
    else:
        try:
            result = order_fix(parse(result).date(),valid_interval)
        except:
            # fail in parse(result)
            status = "BAD_PARSE"
        if not result:
            status = "BAD_PARSE"
    return result, status

In [ ]:
def get_date_from_list(result):
    """Find and return the first date (if any) in a list of strings.
    Used when a cell in a date column has a string with whitespaces.
    Assumption: There is only one date in the list.
    """
    segments = result.strip().split()
    status = "BAD_PARSE"
    if len(segments)>1: # 1 because length 0 is covered in parse_date     
        for segment in segments:
            d = list(datefinder.find_dates(segment.strip()))
            if d:
                result = d[0]
                status = "GOOD_PARSE"
                break
    return result, status

In [ ]:
def parse_date(x,valid_interval):
    """Find and return a date in a cell.
    Steps:
        - Check if NA -> MISSING_VAL 
        - Check if date is in excel 5 digit format -> GOOD_PARSE or BAD_PARSE
        - Check if datefinder can get date -> GOOD_PARSE or BAD_PARSE
        - Check if string split on whitespace has dates -> GOOD_PARSE or BAD_PARSE
    """
    valid_start = valid_interval[0]
    valid_end = valid_interval[1]
    status = "GOOD_PARSE"
    if check_na(x):
        return "NA", "MISSING_VAL"
    result = str(x).strip()
    if result.isnumeric() and 43000 < float(x) < 44500: # Impose loose limits on date
        result = dt.date(1900, 1, 1) + timedelta(float(result)-2) # -2 required due to excel date peculiarity
        if result < valid_start or result > valid_end:
            status = "BAD_PARSE"
    else:
        try:
            result = list(datefinder.find_dates(result.strip()))[0] 
        except:
            status = "BAD_PARSE"
        if status == "BAD_PARSE":
            result, status = get_date_from_list(result)
        if not result:
            status = "BAD_PARSE"
    return result, status

In [ ]:
# Enum parsing
# TO-DO: Set NA introduced by enum map as MISSING_VAL and not GOOD_PARSE - Done, test this functionality

def parse_enum(x,enum_map):
    status = "GOOD_PARSE"
    if check_na(x):
        return "NA", "MISSING_VAL"
    result = x
    result = str(result.strip()).lower()
    if result in enum_map:
        result = enum_map[result]
        if result == "na":
            result, status = "NA", "MISSING_VAL"
    elif len(result) and result.split()[0] in enum_map:
        result = enum_map[result.split()[0]]
    elif len(result) and stem(result.split()[0]) in enum_map:
        result = enum_map[stem(result.split()[0])]
    else:
        status = "BAD_PARSE"
    return result, status


In [ ]:
# Regex parsing

def parse_regex(x,regex_pat):
    result = x
    status ="GOOD_PARSE"
    if check_na(x):
        return "NA", "MISSING_VAL"
    result = str(result)
    if not re.match(regex_pat, result):
        status = "BAD_PARSE"
    return result, status

In [ ]:
def parse_df_and_mark_review(df,col,col_type,parse_args):
    """Parse a column and update the review columns in case there is a parsing issue
        - Update review column (Not needed?)
        - Update column value and individual review column status
    """
    if (col_type =='regex'):
        df['tmp_status'] = df[col].apply(lambda x: parse_regex(x,parse_args)[1])
        df[col] = df[col].apply(lambda x: parse_regex(x,parse_args)[0])
        df['Review'] = df.apply(lambda x: (x['Review'] == True or (x['tmp_status'] == 'BAD_PARSE')), axis=1)
    elif (col_type =='enum'):
        df['tmp_status'] = df[col].apply(lambda x: parse_enum(x,parse_args)[1])
        df[col] = df[col].apply(lambda x: parse_enum(x,parse_args)[0])
        df['Review'] = df.apply(lambda x: (x['Review'] == True or (x['tmp_status'] == 'BAD_PARSE')), axis=1)
    elif (col_type =='date'):
        df['tmp_status'] = df[col].apply(lambda x: parse_date(x,parse_args)[1])
        df[col] = df[col].apply(lambda x: parse_date(x,parse_args)[0])
    
    df[col+'_Review'] = df['tmp_status']

    return df

## Column Categories

In [ ]:
# Regular expression columns

regex_columns =['Unique Code (Do not Fill)', 'ICMR ID', 'SNo',
       'Patient ID/IPD ID', 'Name',  'Contact Number', 'Age'] 

In [ ]:
# Date columns

date_columns = [
    'Date of Admission',
    'Date of Outcome',
    'Date of last test (to be left blank)',
    'Dates of latest positive test (to be left blank)',
    'Date of Sample Collection (FIRST)',
    'Date of Sample Collection (Second)', 
    'Date of Sample Collection (Third)',
    'Date of Sample Collection (Fourth)',
    'Date of Sample Collection (Fifth)', 
    'Date of Sample Collection (Sixth)'
]


In [ ]:
# Enum columns
# TO-DO: Get maps for 'Symptomatic (Y/N)' and 'Current Health Status (Stable/Critical)' columns and add those

enum_columns = [
    'Sex (M/F)',
    'Municipal Ward of patient Residence (to be filled by MCGM)',
    'District of patient residence',
    'Facility where admitted',
    'Current Outcome (Admitted/\n Discharge/\n Death/\n Transfer/\n LAMA/\n DAMA)',
    'In case of Transfer, transferred to which hospital/Facility',
    'Is patient in ICU? (Y/N)',
    'If Critical mention intervention (Nasal O2, Facemask O2, HFNC, NRBM, NIV, Ventilator)',
    'On dialysis (Y/N', 
    'Result of last test (to be left blank)',
    'Sample Collected (Y/N)',
    'Result of Sample (Positive/\n Negative/\n Awaited/\n Inconclusive)',
    'Result of Sample(second)',
    'Result of Sample(Third)',
    'Result of Sample (Fourth)',
    'Result of Sample (Fifth)',
    'Result of Sample (Sixth)'
]

In [ ]:
# Text columns
text_columns = []


## Creating enum maps

In [ ]:
# Creating the rough value lists for human map creation (This set of maps needs to be filled up manually)

# for c in enum_columns:
#     fname = "".join(c.split()) +".csv"
#     fname=fname.replace('/','')
#     tmp = D[c].apply(lambda x: str(x).lower())
#     tmp = sorted(tmp.values)
#     pd.DataFrame(set(tmp)).to_csv(fname,index=False)

In [ ]:
def two_col_file_to_dict(infile):
    df = pd.read_csv(infile,header=0)
    d= pd.Series(df['Canonical'].values,index=df['Value']).to_dict()
    return d

## Domain info setup

In [ ]:
# DOMAIN INFO

# Date - columns
# TODO: future runs -end date will need to be modified 
# TODO: need additional logic later on the dates
valid_date_interval={}
for col in date_columns:
    valid_date_interval[col] = [dt.date(2020,3,10), dt.date(2020,6,10)]

# Enum - columns
enum_map={}

enum_map['Sex (M/F)']=two_col_file_to_dict(GENDER_LIST_FILE)
enum_map['Municipal Ward of patient Residence (to be filled by MCGM)'] = two_col_file_to_dict(WARD_LIST_FILE)
enum_map['District of patient residence'] = two_col_file_to_dict(DISTRICT_LIST_FILE)
enum_map['Facility where admitted'] = two_col_file_to_dict(FACILITY_LIST_FILE)
enum_map['Current Outcome (Admitted/\n Discharge/\n Death/\n Transfer/\n LAMA/\n DAMA)']= two_col_file_to_dict(
    OUTCOME_LIST_FILE)
enum_map['Current Health Status (Stable/Critical)'] = two_col_file_to_dict(HSTATUS_LIST_FILE)
enum_map['If Critical mention intervention (Nasal O2, Facemask O2, HFNC, NRBM, NIV, Ventilator)'] = two_col_file_to_dict(INTERVENTION_LIST_FILE)
enum_map['In case of Transfer, transferred to which hospital/Facility'] = two_col_file_to_dict(FACILITY_LIST_FILE)
enum_map['Result of last test (to be left blank)'] = two_col_file_to_dict(RESULT_STATUS_LIST_FILE)

for col in ['Result of Sample (Positive/\n Negative/\n Awaited/\n Inconclusive)','Result of Sample(second)',
            'Result of Sample(Third)','Result of Sample (Fourth)','Result of Sample (Fifth)','Result of Sample (Sixth)']:
    
    enum_map[col]= two_col_file_to_dict(RESULT_STATUS_LIST_FILE)
    
for col in ['Symptomatic (Y/N)', 'Is patient in ICU? (Y/N)','On dialysis (Y/N', 'Sample Collected (Y/N)']:
    enum_map[col]= two_col_file_to_dict(BOOL_LIST_FILE)

# Regex - columns
# TODO: Fix the patterns
regex_pat={}
regex_pat['Unique Code (Do not Fill)']= '([a-zA-Z]{1,2}[0-9]+)' 
regex_pat['ICMR ID']= '([0-9]+)'
regex_pat['SNo']= '([0-9]+)' 
regex_pat['Patient ID/IPD ID']= '(.*?)'
regex_pat['Name']= '(.*?)'
regex_pat['Age']= '(.*?)'
regex_pat['Contact Number']= '(.*?)'

## Parsing of data

In [ ]:
# date columns
for col in date_columns:
    print(col)
    D=parse_df_and_mark_review(D,col,'date',valid_date_interval[col])

In [ ]:
# enum columns
for col in enum_columns:
    print(col)
    D=parse_df_and_mark_review(D,col,'enum',enum_map[col])
    print(D[col+'_Review'].value_counts())

In [ ]:
# regex columns
for col in regex_columns:
    print(col)
    D=parse_df_and_mark_review(D,col,'regex',regex_pat[col])
    print(D[col+'_Review'].value_counts())

In [ ]:
# text columns ?
# Do we need this or move symptomatic to enum?

## Fixing dates

In [ ]:
# Suspected dates are those for which interchanging the date and month gives a different valid date
suspected_dates = set([(4, 5), (5, 4), (4, 6), (6, 4), (5, 6), (6, 5)
])

In [ ]:
def fix_date(date, valid_interval):
    """
    Flag missing and bad dates.
    For remaining values:
        - Set year as 2020.
        - Check if suspected date.
        - If date is invalid, check if invalidity can be corrected by swapping date and month.
        (Necessary for cases where it is ambiguous whether to parse as mm/dd or dd/mm)
    """
    valid_start = valid_interval[0]
    valid_end = valid_interval[1]
    status = "GOOD_PARSE"
    if date == "NA":
        status = "MISSING_PARSE"
    elif not isinstance(date, dt.date): # and not isinstance(date, dt.datetime)
        status = "BAD_PARSE"
    else:
        date = dt.date(2020, date.month, date.day)
        if (date.day, date.month) in suspected_dates:
            status = "SUSPECTED_PARSE"
        elif ((date > valid_end) or (date < valid_start)):
            try:
                date = dt.date(2020, date.day, date.month)
            except:
                status = "BAD_PARSE"
        else:
            pass
    return date, status

def fix_dates(df, col, col_type, valid_interval):
    df[col+'_Review'] = df[col].apply(lambda x: fix_date(x,valid_interval)[1])
    df[col] = df[col].apply(lambda x: fix_date(x,valid_interval)[0])
    
    df['Review'] = df.apply(lambda x: (
        (x['Review'] == True) or (x[col+'_Review'] == 'BAD_PARSE') or (x[col+'_Review'] == 'SUSPECTED_PARSE')), axis=1)
    
    return df

In [ ]:
for col in date_columns:
    print(col)
    D=fix_dates(D,col,'date',valid_date_interval[col])
    print(D[col+'_Review'].value_counts())

In [ ]:
status_col = []

m = D.shape[0]
n = D.shape[1]

i = 0

for i in range(m):
    if not i%1000:
        print("Processed "+str(i))
    suspect_flag = False
    bad_flag = False
    for j in range(40, n):
        if D.iloc[i,j].startswith("BAD"):
            bad_flag = True
        if D.iloc[i,j].startswith("SUSPECTED"):
            suspect_flag = True
            break
    if suspect_flag:
        status_col.append("SUSPECTED")
    elif bad_flag:
        status_col.append("BAD")
    else:
        status_col.append("GOOD")
    i += 1
        
D['overall_status'] = status_col           

In [ ]:
del D['tmp_status']

## Results

In [ ]:
D.head()

In [ ]:
pd.set_option("display.max_columns", 500)
D.describe()

In [ ]:
# Compile value counts for review columns

D_vc = D.iloc[:,40:]
vc = D_vc.apply(lambda x: x.value_counts()).T.stack()

In [ ]:
# Value counts for review columns
vc

## Writing out data

In [ ]:
timestr = time.strftime("%Y-%m-%d-%H-%M-%S")
D_filename = "".join(['../data/consolidated_wards_clean_',timestr,'.csv'])
# D.to_csv(D_filename, sep=',',index=False) # Copy of latest version 
D.to_csv('../data/consolidated_wards_clean.csv', sep=',',index=False) # Current working file for more processing

In [ ]:
# Creating the rough value lists for human map creation (To get any more values to map from rows with BAD_PARSE cells)

# for c in enum_columns:
#     fname = "".join(c.split()) +".csv"
#     fname=fname.replace('/','')
#     tmp = D.loc[D[c+'_Review'] == 'BAD_PARSE']
#     tmp = tmp[c]
#     print(tmp)
#     tmp = tmp.apply(lambda x: str(x).lower())
#     tmp = sorted(tmp.values)
#     pd.DataFrame(set(tmp)).to_csv('csv/'+fname,index=False)

In [ ]:
# Save value counts to file
vc_filename = "".join(['../data/value_counts_',timestr,'.csv'])
# vc.to_csv(vc_filename, header = ['Value Counts'])